In [1]:
# Brian-setup
from brian2 import BrianLogger, start_scope, NeuronGroup, Synapses, network_operation, run, StateMonitor, SpikeMonitor, stop, rand
from brian2.units import ms, volt, mV, amp, mA, siemens, Hz
from brian2 import figure, grid, axhline, xlabel, ylabel, plot, legend
from brian2tools import brian_plot

from brian2 import prefs, BrianLogger
prefs.logging.file_log_level = 'WARNING'
BrianLogger.file_handler.setLevel(prefs.logging.file_log_level)
# print(BrianLogger.file_handler)

# Scientific libs:
import numpy as np
%matplotlib inline

<FileHandler /tmp/brian_debug__082m4oe.log (WARNING)>


In [2]:
def perform_network_firing_rate_test(a, b, s=10, n=500, tau_g=5*ms, I_ext=0.1*mA, verbose=True, do_plot=True,
                                    debug=False):
    start_scope()

    tau = 1*ms;
    c_inh = -65*mV; d_inh = 2*mV;
    eqs_exc = '''
    dv/dt = ((0.04*v**2)/mV + 5*v + 140*mV - u + (I_syn)*ohm)/tau : volt
    du/dt = a * (b*v - u)/tau : volt

    I_syn : amp
    c : volt
    d : volt
    
    oper_ctr : 1
    '''
    
    eqs_inh = '''
    dv/dt = ((0.04*v**2)/mV + 5*v + 140*mV - u + (I_syn)*ohm)/tau : volt
    du/dt = a * (b*v - u)/tau : volt
    
    I_syn : amp
    '''
    
    n_excit = int(0.8 * n)
    n_inhib = n-n_excit
    if(debug):
        print('n_excit', n_excit)
        print('n_inhib', n_inhib)
    
    G_excit = NeuronGroup(n_excit, eqs_exc, threshold='v>30*mV', 
                    reset='''v=c; u=u+d;''', 
                    method='euler')
    G_excit.c = '-65*mV + 15*(rand()**2)*mV'
    G_excit.d = '8*mV - 6*(rand()**2)*mV'
    G_excit.v = G_excit.c
    G_excit.I_syn = 0*mA
    
    G_inhib = NeuronGroup(n_inhib, eqs_inh, threshold='v>30*mV', 
                    reset='''v=c_inh; u=u+d_inh;''', 
                    method='euler')
    G_inhib.v = c_inh
    G_inhib.I_syn = 0*mA
    
    # variable for simple assert-test
    G_excit.oper_ctr[0] = 0
    
    # synapses: 4 groups
    eqs_syn = '''
    dg/dt = g/(tau_g) : siemens
    '''
#     eqs_syn_i = '''
#     dg/dt = -g/(tau_g*ms) : siemens
#     I_syn : amp
#     '''
    S1 = Synapses(G_excit, G_excit, eqs_syn, on_pre='g_pre=1*siemens; I_syn_post += g_pre * (v_pre - c_post)')
    S2 = Synapses(G_excit, G_inhib, eqs_syn, on_pre='g_pre=1*siemens; I_syn_post += g_pre * (v_pre - c_inh)')
    S3 = Synapses(G_inhib, G_excit, eqs_syn, on_pre='g_pre=1*siemens; I_syn_post -= g_pre * (v_pre - c_post)')
    S4 = Synapses(G_inhib, G_inhib, eqs_syn, on_pre='g_pre=1*siemens; I_syn_post -= g_pre * (v_pre - c_inh)')
    
    # probabilistic method: S.connect(condition='i!=j', p=p_conn)    
    # Paper method; hardcode wiring to 10 (random) neurons, for each neuron
    for ind in range(0, n_excit):
        for _ in range(0, 10):
            target_index = int(rand() * n)
            while(target_index==ind):
                target_index = int(rand() * n)
            if(target_index < n_excit):
                S1.connect(i=ind, j=target_index)
            else:
                S2.connect(i=ind, j=target_index%n_excit)
    
    for ind in range(0, n_inhib):
        for _ in range(0, 10):
            target_index = int(rand() * n)
            while(target_index==ind):
                target_index = int(rand() * n)
            if(target_index < n_excit):
                S3.connect(i=ind, j=target_index)
            else:
                S4.connect(i=ind, j=target_index%n_excit)
    
    
    # random neuron excitation at 100 micro-Ampere each timestep
    @network_operation(dt=tau)
    def excite_one_random_neuron():
#         # clear previous external stimulus
#         G_excit.I_ext = 0
#         G_inhib.I_ext = 0
        
        idx = int(rand()*(n-1))
        if(idx<n_excit):
            G_excit.I_syn[idx] += I_ext
        else:
            G_inhib.I_syn[idx%n_excit] += I_ext

        G_excit.oper_ctr[0] += 1
    
    
    def assert_network_op_post_run(t):
        assert(G_excit.oper_ctr[0] == t)
        
        if(debug):
            g_active = []
            for g_i in G_excit.g:
                if g_i>0.0001*siemens:
                    g_active += [g_i]
            for g_i in G_inhib.g:
                if g_i<-0.0001*siemens:
                    g_active += [g_i]
            print('t='+str(t)+', # recently spiking neurons: ', len(g_active))
            if(len(g_active) < 10):
                print(g_active)

    # run for 1 second
    t1=1000
    run(t1*tau)
    assert_network_op_post_run(t1)

    # start recording spikes
    statemon_excit = StateMonitor(G_excit[:], 'v', record=True)
    spikemon_excit = SpikeMonitor(G_excit[:], variables='v')
    statemon_inhib = StateMonitor(G_inhib[:], 'v', record=True)
    spikemon_inhib = SpikeMonitor(G_inhib[:], variables='v')

    t2=8000
    run(t2*tau)
    assert_network_op_post_run(t1+t2)
    
    avg_neuron_firing_rate_excit = spikemon_excit.num_spikes/(n*t2*tau)
    avg_neuron_firing_rate_inhib = spikemon_inhib.num_spikes/(n*t2*tau)
    
    chaotic_behaviour = avg_neuron_firing_rate_inhib > 1000.*Hz or avg_neuron_firing_rate_inhib > 1000.*Hz
    if(do_plot and not chaotic_behaviour):
        # create spike plots
        figure(figsize=(9, 4))
        grid(True)
        axhline(30, ls='-', c='lightgray', lw=3)
        plot(statemon_excit.t/ms, statemon_excit.v.T/mV, '-')
        plot(statemon_inhib.t/ms, statemon_inhib.v.T/mV, '-')
        xlabel('Time (ms)')
        ylabel('v (mV)');
        brian_plot(spikemon_excit)
        brian_plot(spikemon_inhib);
        
    if(debug):
        print('#spikes; exc: {}, inh: {}'.format(spikemon_excit.num_spikes, spikemon_inhib.num_spikes))
        
    stop()
        
    return [avg_neuron_firing_rate_excit, avg_neuron_firing_rate_inhib]

In [3]:
import datetime as dt
def write_to_logfile(params, log_str, opt_fname_postfix=''):
    fname = 'v4_results'
    if(opt_fname_postfix!=''):
        fname += '_' + opt_fname_postfix
    fname += '.txt'
    
    prefix = '[{}] (a={}, b={}, s={}, n={}, tau_g={}, I={})'.format(dt.datetime.now(), params[0], params[1], params[2], params[3], params[4], params[5])
    full_str = prefix + ' ' + log_str + '\n'
    with open(fname, 'a') as f:
        f.write(full_str)

In [4]:
def avg_network_firing_rate_over_N_runs(a, b, s=10, n=500, tau_g=5*ms, I_ext=0.1*mA, verbose=False, do_plot=False, N=30,
                                       debug=False, log_fname_postfix=''):
    parameters = [a, b, s, n, tau_g, I_ext]
    avg_excits = []
    avg_inhibs = []
    
    log_str = 'Running experiment over N={} runs for parametrisation: a={}, b={}, s={}, n={}, tau_g={}. with I={}'.format(N, a, b, s, n, tau_g, I_ext)
    if(verbose):
        print(log_str)
    write_to_logfile(parameters, log_str, log_fname_postfix)
    
    for ctr in range(0, N):
        [cur_avg_e, cur_avg_i] = perform_network_firing_rate_test(a, b, s, n, tau_g, I_ext, verbose, do_plot, debug)
        avg_excits += [cur_avg_e]
        avg_inhibs += [cur_avg_i]
        
        log_str = 'run #{}, cur_avg_e_rate: {}, cur_avg_i: {}'.format(ctr, cur_avg_e, cur_avg_i)
        write_to_logfile(parameters, log_str, log_fname_postfix)
        if(verbose):
            print(log_str)
    
    mean_e = np.mean(avg_excits)
    std_e = np.std(avg_excits)
    mean_i = np.mean(avg_inhibs)
    std_i = np.std(avg_inhibs)
    
    log_str = 'mean_e: {} std_e: {} mean_i: {} std_i {}'.format(mean_e, std_e, mean_i, std_i)
    write_to_logfile(parameters, log_str, log_fname_postfix)
    print('parameters: {}, log_str: {}'.format(parameters, log_str))
    return [mean_e, std_e, mean_i, std_i]